In [3]:
import tensorflow as tf
from tensorflow.contrib import rnn

import pandas as pd
import numpy as np
import spacy

nlp = spacy.load('en')

dtypes ={
    'id': np.uint32,
    'qid1': np.uint32,
    'qid2': np.uint32,
    'question1': np.str,
    'question2': np.str,
    'is_duplicate': np.uint8
}

df = pd.read_csv('../../dataset/quora_train.csv.zip', dtype=dtypes, compression='zip', usecols=['question1', 'question2', 'is_duplicate'], nrows=1000)

In [4]:
def pad(v):
    zeros = np.zeros([100, 300])
    zeros[:v.shape[0],:v.shape[1]] = v
    return zeros
    
q1s = df['question1'].map(nlp).map(lambda x: np.matrix([t.vector for t in x])).map(pad)
q2s = df['question2'].map(nlp).map(lambda x: np.matrix([t.vector for t in x])).map(pad)

q1m = np.concatenate(q1s.values).flatten().reshape([-1, 100, 300])
q2m = np.concatenate(q2s.values).flatten().reshape([-1, 100, 300])

labels = df['is_duplicate'].values

In [5]:
train_q1m, test_q1m = q1m[:4000], q1m[4000:]
train_q2m, test_q2m = q2m[:4000], q2m[4000:]
train_labels, test_labels = labels[:4000], labels[4000:]

In [6]:
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

tf.reset_default_graph()

Q1 = tf.placeholder(tf.float32, shape=[None, 100, 300])
Q2 = tf.placeholder(tf.float32, shape=[None, 100, 300])
batch_size = tf.placeholder(tf.int32)
is_dup = tf.placeholder(tf.uint8, shape=[None])
dup_oh = tf.one_hot(is_dup, 2)

cell1 = rnn.GRUCell(64)
cell2 = rnn.GRUCell(64, reuse=True)

state = cell1.zero_state(batch_size, dtype=tf.float32)

#Q1t = tf.transpose(Q1, [1, 0, 2])
#Q2t = tf.transpose(Q2, [1, 0, 2])

outputs1, _1= tf.nn.dynamic_rnn(cell1, Q1, sequence_length=length(Q1), initial_state=state)
outputs2, _2= tf.nn.dynamic_rnn(cell2, Q2, sequence_length=length(Q2), initial_state=state)

output1 = tf.reduce_mean(outputs1, 1)
output2 = tf.reduce_mean(outputs2, 1)

out = tf.concat([output1, output2], axis=1)

In [7]:
keep_prob = tf.placeholder(tf.float32)
out = tf.nn.dropout(out, keep_prob=keep_prob)

#W1 = tf.get_variable(shape=[512, 256], initializer=tf.contrib.layers.xavier_initializer(), name='weight1')
#b1 = tf.get_variable(shape=[256], initializer=tf.contrib.layers.xavier_initializer(), name='bias1')
#y1 = tf.nn.relu(tf.matmul(out, W1) + b1)
#y1 = tf.nn.dropout(y1, keep_prob=keep_prob)

W2 = tf.get_variable(shape=[128, 2], initializer=tf.contrib.layers.xavier_initializer(), name='weight2')
b2 = tf.get_variable(shape=[2], initializer=tf.contrib.layers.xavier_initializer(), name='bias2')
y2 = tf.matmul(out, W2) + b2

In [8]:
cost = tf.nn.softmax_cross_entropy_with_logits(logits=y2, labels=dup_oh)
cost = tf.reduce_mean(cost)

optimizer = tf.train.AdamOptimizer(learning_rate=0.003).minimize(cost)

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [10]:
from tqdm import tqdm

train_dict = {Q1: train_q1m, Q2: train_q2m, is_dup: train_labels, keep_prob: 0.3, batch_size: train_q1m.shape[0]}
test_dict = {Q1: test_q1m, Q2: test_q2m, is_dup: test_labels, keep_prob: 1.0, batch_size: test_q1m.shape[0]}

for i in tqdm(range(10000)):
    sess.run(optimizer, feed_dict=train_dict)
    if i % 10 == 0:
        print(sess.run(cost, feed_dict=train_dict), sess.run(cost, feed_dict=test_dict))

  0%|          | 0/10000 [00:00<?, ?it/s]

ResourceExhaustedError: OOM when allocating tensor with shape[1000,64]
	 [[Node: GRUCellZeroState/zeros = Fill[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](GRUCellZeroState/stack, GRUCellZeroState/zeros/Const)]]
	 [[Node: rnn_1/Shape_1/_33 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1157_rnn_1/Shape_1", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'GRUCellZeroState/zeros', defined at:
  File "/home/ic/.pyenv/versions/3.6.1/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ic/.pyenv/versions/3.6.1/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ic/.pyenv/versions/3.6.1/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ic/.py

In [9]:
variables_names =[v.name for v in tf.trainable_variables()]
values = sess.run(variables_names)
for k,v in zip(variables_names, values):
    print(k, v)

rnn/basic_lstm_cell/weights:0 [[ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 ..., 
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]]
rnn/basic_lstm_cell/biases:0 [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan 

In [9]:
output1

<tf.Tensor 'Mean:0' shape=(100, 128) dtype=float32>

In [20]:
state

<tf.Tensor 'GRUCellZeroState/zeros:0' shape=(100, 256) dtype=float32>

In [21]:
output1

<tf.Tensor 'Mean:0' shape=(100, 256) dtype=float32>

Dimension(None)